In [1]:
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils

In [26]:
import sys
import json
import time
import websocket
import ssl
import socks  # Provided by PySocks
import socket

def fetch_nostr_events(url, network, start, stop):
    # Generate a random subscription ID
    events = []
    sub_id = "" + str(int(time.time()))

    # Filter to get all events between start and stop
    req = [
        "REQ",
        sub_id,
        {
            "since": int(start),
            "until": int(stop)
        }
    ]

    close_req = ["CLOSE", sub_id]

    # Configure proxy if using Tor
    if network.lower() == "tor":
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
        socket.socket = socks.socksocket
        sslopt = {"cert_reqs": ssl.CERT_NONE}
    else:
        sslopt = {"cert_reqs": ssl.CERT_REQUIRED}

    def on_message(ws, message):
        data = json.loads(message)
        if data[0] == "EVENT":
            try:
                Event.from_dict(data[2])
            except Exception as e:
                print(data[2])
            events.append(data[2])
        elif data[0] == "EOSE":
            print("End of stored events.")
            ws.send(json.dumps(close_req))
            ws.close()

    def on_error(ws, error):
        print("Error:", error)

    def on_close(ws, close_status_code, close_msg):
        print("Connection closed.")


    def on_open(ws):
        ws.send(json.dumps(req))

    print(f"Connecting to {url} over {network}...")
    ws = websocket.WebSocketApp(
        url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )

    ws.run_forever(sslopt=sslopt)
    print("Connection closed.")
    print(f"Fetched {len(events)} events.")
    return events

# Example usage:
events = fetch_nostr_events("wss://nostr.lnwallet", "clearnet", 0, 1746895719)

Connecting to wss://nostr.lnwallet over clearnet...
Error: [Errno -2] Name or service not known
Connection closed.
Connection closed.
Fetched 0 events.


In [13]:
time.time()

1746895719.1371865

In [10]:
async def ping_relay(relay_url):
    rtt_ms = None
    try:
        start = time.time()
        async with websockets.connect(relay_url) as ws:
            end = time.time()
            rtt_ms = int((end - start) * 1000)
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(f"Error connecting to {relay_url}: {e}")
    return rtt_ms

def fetch_relay_info(url):
    headers = {'Accept': 'application/nostr+json'}
    data = None
    error = None
    relay_url = url[6:] if url.startswith('wss://') else url[5:] if url.startswith('ws://') else url
    try:
        response = requests.get(f"https://{relay_url}", headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
        else:
            error = response.text
    except Exception:
        try:
            response = requests.get(f"http://{relay_url}", headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
            else:
                error = response.text
        except Exception as e:
            error = str(e)
    return url, {'data': data, 'error': error}

rtt_ms = await ping_relay('wss://relay.damus.io')
url, info = fetch_relay_info('wss://relay.damus.io')

Error connecting to wss://relay.damus.io: name 'websockets' is not defined


In [ ]:
relays = pd.read_csv('../data/raw/relays_url.csv')
for url in relays['url']:
    try:
        async with websockets.connect(url) as ws:
            print(f'Connected to {url}')
    except Exception as e:
        print(f'Failed to connect to {url}: {e}')
    finally:
        print('Done')

In [ ]:
RELAY_URL = "wss://relay.nostrdice.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://relay.boroghor.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://schnorr.me"
RELAY_URL = "wss://mastodon.cloud/api/v1/streaming"
async def fetch_events(from_ts: int, to_ts: int):
    events = []
    subscription_id = str(uuid.uuid4())[:64]
    filter_obj = {
        # "since": from_ts,
        # "until": to_ts,
        # "limit": 10
    }

    async with websockets.connect(RELAY_URL) as ws:
        print(f"Connesso a {RELAY_URL}")

        req_msg = ["REQ", subscription_id, filter_obj]
        await ws.send(json.dumps(req_msg))
        print(f"Inviato: {req_msg}")

        try:
            while True:
                raw_message = await asyncio.wait_for(ws.recv(), timeout=30)
                message = json.loads(raw_message)

                if not isinstance(message, list):
                    print("Messaggio non valido (non è un array JSON)")
                    continue

                msg_type = message[0]

                if msg_type == "EVENT":
                    _, sub_id, event = message
                    if sub_id == subscription_id:
                        # print(f"Ricevuto evento: {event}")
                        events.append(event)

                elif msg_type == "EOSE":
                    _, sub_id = message
                    if sub_id == subscription_id:
                        print("Fine degli eventi storici.")
                        break

                elif msg_type == "NOTICE":
                    _, notice = message
                    print(f"NOTICE dal relay: {notice}")

                elif msg_type == "CLOSED":
                    _, sub_id, reason = message
                    if sub_id == subscription_id:
                        print(f"Subscription chiusa dal relay: {reason}")
                        break

                elif msg_type == "OK":
                    _, event_id, accepted, message_str = message
                    status = "accettato" if accepted else "rifiutato"
                    print(f"Evento {event_id} {status} - {message_str}")

                else:
                    print(f"Messaggio sconosciuto: {message}")

        except asyncio.TimeoutError:
            print("Timeout: nessun messaggio ricevuto per 30 secondi.")
        finally:
            close_msg = ["CLOSE", subscription_id]
            await ws.send(json.dumps(close_msg))
            print("Subscription chiusa.")
    return events

# Esegui in una cella async
async def main():
    to_time = int(time.time())
    from_time = to_time - 600
    return await fetch_events(0, 1727090175)

events = await main()
